# Pubmed data acquisition and munging
- Download pubmed data and convert it into a handy csv.

## Note on downloading PubMed data.
The easiest way I have found to do this, is to simply go here: ftp://ftp.ncbi.nlm.nih.gov/pubmed/baseline
... and then download everything. With windows, you can use Windows Subsystem for Linux (WSL), or you can just use a terminal on Mac/Linux.

Replace 'destinationPath' with the path to an empty directory where you want to download the data to. In my case, this was `D:\pubmed\data`

`wget ftp://ftp.ncbi.nlm.nih.gov/pubmed/baseline/* destinationPath`

In [11]:
p = os.path.abspath(r'D:\pubmed\data\*')
filepaths = glob.glob(p)
len(filepaths)

In [12]:
filepaths = filepaths[::-1]
len(filepaths)

In [13]:
# build a dataframe

# columns
articletitles = []
titles = []
abstracts = []
articledates = []
reviseddates = []
years = []
dois = []
piis = []
pmcs = []
pmids = []

years_in_file = set()
# years_of_interest = set([str(x) for x in range(2015,2021)])
years_of_interest = {'2020','2019'}



for i,filepath in enumerate(filepaths):
    
    if len(list(years_in_file))>0:
        if not any([(x in years_of_interest) for x in list(years_in_file) ]):
            break
    
    f = gzip.open(filepath)
    soup = bs(f.read(),'lxml')
    years_in_file = set()
    articles = soup.find_all('pubmedarticle')
    for article in articles:
        journal = None
        articletitle = None
        title = None
        abstract = None
        articledate = None
        reviseddate = None
        year = None
        doi = None
        pii =None
        pmid = None
        pmc = None
        try:
            year = article.pubdate.year.text
            years_in_file.add(year)
        except:
            pass
        try:
            articledate = article.articledate.year.text+'-'+article.articledate.month.text+'-'+article.articledate.day.text
            
        except:
            pass
        try:
            articletitle = article.articletitle.text
            
        except:
            pass
        try:
            abstract = article.abstract.text
            
        except:
            pass
        try:
            title = article.title.text
            
        except:
            pass
        try:
            reviseddate = article.daterevised.year.text+'-'+article.daterevised.month.text+'-'+article.daterevised.day.text
            reviseddates.append(reviseddate)
        except:
            pass
        try:
            ids = {x.attrs['idtype']: x.text  for x in article.find_all('articleid')}
            try:
                doi = ids['doi']
            except:
                pass
            try:
                pmid = article.pmid.text
            except:
                try:
                    pmid = ids['pmid']
                except:
                    pass
            try:
                pii = ids['pii']
            except:
                pass
            try:
                pmc = ids['pmc']
            except:
                pass
        except:
            pass
        
        years.append(year)
        articledates.append(articledate)
        articletitles.append(articletitle)
        abstracts.append(abstract)
        titles.append(title)
        dois.append(doi)
        pmids.append(pmid)
        pmcs.append(pmc)
        piis.append(pii)
    if i==34:
        break
    print(datetime.datetime.now(), ' | ', i,' files done | ',len(years),' articles done')
print('Done.')

In [14]:
df = pd.DataFrame({'title':titles,
                  'articletitle':articletitles,
                  'abstract':abstracts,
                  'year':years,
                  'articledate':articledates,
                  'doi':dois,
                  'pmid':pmids,
                  'pii':piis,
                  'pmc':pmcs,
                  })
df.shape

In [ ]:
df.to_csv('data/pubmed_sample.csv')